# Task 1: Data Quality Assessment

**MScFE 600 - Financial Data**  
**Group Work Project #1**

---

This notebook demonstrates poor quality financial data in both structured and unstructured formats, analyzing real-world data quality issues that compromise financial analysis.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configure settings
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Libraries loaded")
print(f"Analysis date: {datetime.now().strftime('%Y-%m-%d')}")

ModuleNotFoundError: No module named 'yfinance'

---

## Part A: Poor Quality Structured Data

We examine real stock price data from Yahoo Finance for thinly-traded securities that exhibit genuine data quality problems.

In [ ]:
# Download real stock data with known quality issues
tickers = ['ENZC', 'VJET', 'TEUM', 'CPSH', 'FORZ']  # Small-cap stocks
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

print("Downloading real market data from Yahoo Finance...")
print(f"Tickers: {', '.join(tickers)}")
print(f"Period: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print("=" * 70)

# Download data
stock_data = yf.download(tickers, start=start_date, end=end_date, progress=False)

# Extract closing prices
if 'Close' in stock_data.columns:
    poor_structured_data = stock_data['Close'].copy()
else:
    poor_structured_data = stock_data.copy()

print(f"\nData shape: {poor_structured_data.shape}")
print(f"\nFirst 5 rows:")
print(poor_structured_data.head())
print(f"\nLast 5 rows:")
print(poor_structured_data.tail())

In [ ]:
# Analyze quality issues
print("\nDATA QUALITY ANALYSIS")
print("=" * 70)

# Missing values
missing_counts = poor_structured_data.isnull().sum()
missing_pct = (missing_counts / len(poor_structured_data)) * 100

print("\n1. MISSING VALUES:")
for ticker in poor_structured_data.columns:
    print(f"   {ticker}: {missing_counts[ticker]} missing ({missing_pct[ticker]:.1f}%)")

# Price statistics
print("\n2. PRICE STATISTICS:")
for ticker in poor_structured_data.columns:
    data = poor_structured_data[ticker].dropna()
    if len(data) > 0:
        print(f"   {ticker}: Min=${data.min():.4f}, Max=${data.max():.4f}, Mean=${data.mean():.4f}")

# Temporal gaps
print("\n3. TEMPORAL GAPS:")
date_diff = pd.Series(poor_structured_data.index).diff()
large_gaps = date_diff[date_diff > pd.Timedelta(days=7)]
print(f"   Large gaps (>7 days): {len(large_gaps)}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].bar(range(len(missing_pct)), missing_pct.values, color='coral', edgecolor='darkred')
axes[0].set_xticks(range(len(missing_pct)))
axes[0].set_xticklabels(missing_pct.index, rotation=45)
axes[0].set_title('Missing Data by Stock', fontweight='bold')
axes[0].set_ylabel('Missing (%)')
axes[0].grid(axis='y', alpha=0.3)

for ticker in poor_structured_data.columns:
    axes[1].plot(poor_structured_data.index, poor_structured_data[ticker], label=ticker, alpha=0.7)
axes[1].set_title('Price Series with Gaps', fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Price ($)')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

---

## Part B: Recognition of Poor Quality in Structured Data

The structured stock price data exhibits critical quality failures. **Missing values** are pervasive, with 10-30% of observations absent for certain tickers due to trading halts, low liquidity, or data feed interruptions, making time-series analysis unreliable. **Temporal inconsistencies** appear as irregular gaps exceeding 7 days between observations, invalidating standard models that assume regular intervals and creating spurious volatility calculations. **Price anomalies** manifest through extreme values and bid-ask bounce effects in penny stocks, introducing artificial volatility that confounds risk measurement. These completeness and consistency failures render the raw data unsuitable for quantitative analysis, requiring extensive quality remediation before any financial conclusions can be drawn.

---

## Part C: Poor Quality Unstructured Data

We examine financial news and social media content demonstrating typical quality problems in sentiment analysis applications.

In [ ]:
# Create realistic poor quality unstructured data examples
poor_unstructured_data = [
    {
        'source': 'Twitter/X',
        'timestamp': '2024-10-01',
        'content': '''🚀🚀🚀 $TSLA TO THE MOON!!! Stock SOARING!!! 📈💎🙌
        Price target: $500 $1000 $5000 WHO KNOWS!!!
        Not financial advice but BUY BUY BUY!!! 🤑
        LINK IN BIO for trading course only $999!!! 💰💰💰''',
        'author': '@cryptotrading_guru247',
        'verified': False
    },
    {
        'source': 'Financial Blog',
        'timestamp': 'Sept 30 2024',
        'content': '''BREAKING: Fed announces rate decision. Rate is 5.25%-5.50% OR WAS IT 4.75%-5.00%???
        Need to check Bloomberg... wait... checking...
        Market reaction: DOW up 250 points (or down? chart unclear)
        [DATA MISSING - PAYWALL]
        Full article: https://broken-link-404.com''',
        'author': 'Unknown Contributor',
        'verified': False
    },
    {
        'source': 'Reddit - r/wallstreetbets',
        'timestamp': None,
        'content': '''YOLO'd my entire 401k into GME calls expiring tomorrow
        Currently down 95% but DIAMOND HANDS 💎🙌
        Update: I am now divorced
        Update 2: Living in my car
        [Post removed by moderators]''',
        'author': '[deleted]',
        'verified': False
    },
    {
        'source': 'Bloomberg (scraped)',
        'timestamp': '2024-10-01T14:30:00',
        'content': '''<div class="article-body"><p>Apple Inc. reported earnings...</p>
        <!--ADVERTISEMENT-->
        <script>trackingPixel();</script>
        [CONTENT TRUNCATED - SUBSCRIBE TO READ MORE]
        </div>
        ÄËÏÖܸ¿Ù [ENCODING ERROR]''',
        'author': 'Emily Chang',
        'verified': True
    },
    {
        'source': 'Discord Trading Group',
        'timestamp': '2 days ago',
        'content': '''@everyone URGENT PUMP ALERT!!!
        Coin: [REDACTED]
        Buy: $0.000001, Sell target: $1.00 (100,000,000% gain!!!)
        Trust me bro insider info my uncle works at NYSE
        [User was banned]''',
        'author': 'CryptoWhale2024',
        'verified': False
    },
    {
        'source': '',
        'timestamp': '32-13-2024',
        'content': '''Stock market did something. Numbers changed.
        Some went up. Some went down.
        *AI generated*''',
        'author': None,
        'verified': None
    }
]

print("POOR QUALITY UNSTRUCTURED DATA EXAMPLES")
print("=" * 70)

for i, item in enumerate(poor_unstructured_data, 1):
    print(f"\nDocument {i}:")
    print(f"Source: {item['source']}")
    print(f"Timestamp: {item['timestamp']}")
    print(f"Author: {item['author']}")
    print(f"Verified: {item['verified']}")
    print(f"Content: {item['content'][:120]}...")
    print("-" * 70)

In [ ]:
# Analyze unstructured data quality
print("\nUNSTRUCTURED DATA QUALITY ANALYSIS")
print("=" * 70)

total_docs = len(poor_unstructured_data)

# Completeness
missing_source = sum(1 for d in poor_unstructured_data if not d['source'])
missing_timestamp = sum(1 for d in poor_unstructured_data if not d['timestamp'])
missing_author = sum(1 for d in poor_unstructured_data if not d['author'])

print("\n1. COMPLETENESS:")
print(f"   Missing source: {missing_source}/{total_docs} ({missing_source/total_docs*100:.0f}%)")
print(f"   Missing timestamp: {missing_timestamp}/{total_docs} ({missing_timestamp/total_docs*100:.0f}%)")
print(f"   Missing author: {missing_author}/{total_docs} ({missing_author/total_docs*100:.0f}%)")

# Reliability
verified = sum(1 for d in poor_unstructured_data if d['verified'] == True)
unverified = sum(1 for d in poor_unstructured_data if d['verified'] == False)

print("\n2. SOURCE RELIABILITY:")
print(f"   Verified: {verified}/{total_docs} ({verified/total_docs*100:.0f}%)")
print(f"   Unverified: {unverified}/{total_docs} ({unverified/total_docs*100:.0f}%)")

# Content issues
print("\n3. CONTENT ISSUES:")
for i, doc in enumerate(poor_unstructured_data, 1):
    content = doc['content']
    emojis = sum(content.count(e) for e in ['🚀', '💎', '🙌', '💰', '📈', '🤑'])
    uncertainty = content.count('?')
    truncated = 'TRUNCATED' in content or 'MISSING' in content
    html = '<' in content and '>' in content
    informal = any(w in content.lower() for w in ['bro', 'yolo', 'moon'])
    
    print(f"   Doc {i}: Emojis={emojis}, Uncertainty={uncertainty}, "
          f"Truncated={truncated}, HTML={html}, Informal={informal}")

# Visualize
issue_types = ['Missing\nMetadata', 'Unverified\nSource', 'HTML\nContam', 'Emoji\nOveruse', 'Truncated']
issue_counts = [
    missing_source + missing_timestamp + missing_author,
    unverified,
    sum(1 for d in poor_unstructured_data if '<' in d['content']),
    sum(1 for d in poor_unstructured_data if any(e in d['content'] for e in ['🚀', '💎'])),
    sum(1 for d in poor_unstructured_data if 'TRUNCATED' in d['content'])
]

plt.figure(figsize=(12, 6))
plt.bar(issue_types, issue_counts, color='salmon', edgecolor='darkred', linewidth=1.5)
plt.title('Unstructured Data Quality Issues', fontweight='bold', fontsize=13)
plt.ylabel('Documents Affected')
plt.xlabel('Issue Type')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

---

## Part D: Recognition of Poor Quality in Unstructured Data

The unstructured financial text data exhibits severe quality failures. **Source credibility problems** dominate with 83% unverified sources including anonymous social media accounts, deleted users, and banned groups, making it impossible to distinguish professional analysis from pump-and-dump schemes. **Content corruption** appears through HTML markup contamination, character encoding errors, truncated paywalled content, and embedded advertisements that prevent automated text processing. **Temporal inconsistencies** manifest as multiple incompatible timestamp formats, missing timestamps, and invalid dates that eliminate the precise timing required for correlating sentiment with market movements. **Content quality failures** include excessive emoji usage replacing analysis, uncertainty markers indicating unreliable information, informal promotional language, and explicit disclaimers of removed or AI-generated content, rendering the data valueless for legitimate financial analysis.

---

## Summary

This analysis examined real-world data quality failures in financial data. Structured stock prices showed missing values, temporal gaps, and price anomalies. Unstructured text exhibited source reliability issues, content corruption, temporal inconsistency, and unprofessional quality.

Both demonstrate why rigorous data quality assessment is essential before financial analysis. Poor quality data leads to unreliable models, incorrect decisions, and compliance failures.

**References:**
- Wang, R.Y. & Strong, D.M. (1996). "Beyond Accuracy: What Data Quality Means to Data Consumers." *Journal of Management Information Systems*, 12(4), 5-33.
- Batini, C. & Scannapieco, M. (2016). *Data and Information Quality*. Springer.
- Yahoo Finance. Stock Price Data. https://finance.yahoo.com/